## Descripcion de especies por laguna 

Graficos y tablas que resaltan las composiciones faunisticas de cada laguna

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
hernandez = pd.read_csv("dominancia/dominancia_total_Hernandez.csv")
hernandez = hernandez.sort_values(by="Porcentaje_dominancia", ascending=False)
hernandez.head(50)

,Especie,Abundancia_total,Porcentaje_dominancia
0,Nauplios,403.333333,8.705036
1,Copepoditos,250.000000,5.395683
2,Copepodo calanoideo,240.000000,5.179856
3,Plationus patulus,220.000000,4.748201
4,Lecane luna,203.333333,4.388489
5,Chydorido sp 1,196.666667,4.244604
6,Cydorido sp 2. (Chiquito),190.000000,4.100719
7,Bdelloideo,186.666667,4.028777
8,Lecane luna,150.000000,3.237410
9,Bdelloideos,113.333333,2.446043


In [13]:
# Carpeta donde están los CSV (puede ser "." si es el mismo directorio del script)
DATA_DIR = Path("listos/")
OUTPUT_DIR = Path("composicion/")


# Mapeo estación -> nombre de archivo
archivos_estaciones = {
    "Invierno": "invierno.csv",
    "Verano": "verano.csv",
    "Primavera": "primavera.csv",
    "Otoño": "otoño.csv",   # si tu archivo se llama "otono.csv", cambia aquí
}

# Lista de lagunas (según las columnas de tus archivos)
lagunas = [
    "Termas",
    "Viale",
    "Inta",
    "Gazano",
    "Crespo",
    "Hernandez",
    "Espinillo",
]

def procesar_estacion(nombre_estacion, nombre_archivo, guardar_csv=True):
    ruta = DATA_DIR / nombre_archivo
    if not ruta.exists():
        print(f"\n⚠️ Archivo NO encontrado para {nombre_estacion}: {ruta}")
        return

    print(f"\n==============================")
    print(f"  Estación: {nombre_estacion}   ")
    print(f"  Archivo:  {ruta}")
    print(f"==============================\n")

    # Leer CSV
    df = pd.read_csv(ruta)

    # Renombrar la primera columna a "Especie"
    primera_col = df.columns[0]
    df = df.rename(columns={primera_col: "Especie"})

    # Limpiar espacios en los nombres de columnas
    df.columns = [c.strip() for c in df.columns]

    # Opcional: eliminar columnas de grupos grandes si no las querés usar
    columnas_excluir = {"Rotiferos", "Copepodos", "Cladoceros"}
    columnas_usables = [c for c in df.columns if c not in columnas_excluir]

    df = df[columnas_usables]

    for laguna in lagunas:
        
        col_l1 = f"{laguna} L1"
        col_c  = f"{laguna} C"
        col_l2 = f"{laguna} L2"

        # Verificar que existan las columnas (por si alguna laguna no se muestreó en cierta estación)
        if not all(col in df.columns for col in [col_l1, col_c, col_l2]):
            print(f"  ⚠️ Laguna {laguna}: faltan columnas (L1/C/L2). Se omite.\n")
            continue

        # Calcular abundancia total por especie en la laguna (suma de las 3 capas)
        abund_total = (
            df[col_l1].fillna(0) +
            df[col_c].fillna(0) +
            df[col_l2].fillna(0)
        )

        comp = pd.DataFrame({
            "Especie": df["Especie"],
            "Abundancia_total": abund_total
        })

        # Filtrar especies con abundancia > 0
        comp = comp[comp["Abundancia_total"] > 0].copy()

        if comp.empty:
            print(f"  Laguna {laguna}: sin especies registradas (abundancia total = 0).\n")
            continue

        # Calcular porcentaje relativo
        total_laguna = comp["Abundancia_total"].sum()
        comp["Porcentaje"] = comp["Abundancia_total"] / total_laguna * 100

        # Ordenar de mayor a menor
        comp = comp.sort_values("Porcentaje", ascending=False).reset_index(drop=True)

        # Imprimir un resumen en pantalla
        print(f"▶ Laguna: {laguna} — {nombre_estacion}")
        print(f"   Total abundancia (suma de ind/L en L1+C+L2): {total_laguna:.2f}")
        print("   Especies (top 10 por porcentaje):")
        for i, row in comp.head(10).iterrows():
            print(f"    {i+1:2d}. {row['Especie']} — {row['Porcentaje']:.1f}%")
        print("")

        import os

        if guardar_csv:
            os.makedirs(OUTPUT_DIR, exist_ok=True)   # crea carpeta si no existe
            out_name = os.path.join(
                OUTPUT_DIR,
                f"composicion_{laguna}_{nombre_estacion}.csv".replace(" ", "_")
            )
            comp.to_csv(out_name, index=False)


def main():
    for estacion, archivo in archivos_estaciones.items():
        procesar_estacion(estacion, archivo, guardar_csv=True)

if __name__ == "__main__":
    main()


  Estación: Invierno   
  Archivo:  listos\invierno.csv

▶ Laguna: Termas — Invierno
   Total abundancia (suma de ind/L en L1+C+L2): 1038.89
   Especies (top 10 por porcentaje):
     1. nauplios  — 25.0%
     2. Copepodo Cyclopoideo sp 1 — 23.5%
     3. Lepadella cf. ovalis — 15.4%
     4. Copepodito  — 14.8%
     5. Keratella tropica — 5.6%
     6. Pompholix triloba  — 5.5%
     7. Lecane bulla  — 5.0%
     8. Moina sp1 — 1.9%
     9. Bdelloideos  — 1.3%
    10. Lepadella patella  — 1.1%

▶ Laguna: Viale — Invierno
   Total abundancia (suma de ind/L en L1+C+L2): 1410.00
   Especies (top 10 por porcentaje):
     1. Lecane luna — 52.2%
     2. Lecane bulla  — 16.5%
     3. Keratella tropica — 15.7%
     4. Mytilina ventralis   — 5.2%
     5. Polyarthra sp. — 4.0%
     6. nauplios  — 2.1%
     7. Diaphanosoma birgei — 1.9%
     8. Copepodito  — 0.7%
     9. Moina sp1 — 0.6%
    10. Trichocerca cf. grandis  — 0.4%

▶ Laguna: Inta — Invierno
   Total abundancia (suma de ind/L en L1+C+L2):

# Grafico de barras horizontales

In [ ]:
INPUT_DIR = Path("composicion")
PLOTS_DIR = Path("plots")
PLOTS_DIR.mkdir(exist_ok=True)

ESTACIONES = ["Invierno", "Verano", "Primavera", "Otoño"]
LAGUNAS = ["Termas"]
TOP_N = 20

# "Viale", "Inta", "Gazano", "Crespo", "Hernandez", "Espinillo"]

def plot_horizontal_bars(ax, especies, valores, titulo, xmax=None):
    
    """Barras horizontales con el nombre al final de la barra."""
    y = np.arange(len(especies))
    ax.barh(y, valores,edgecolor="black", linewidth=0.8)
    ax.set_yticks([])

    # si no viene xmax, lo calculo para este subplot
    if xmax is None:
        xmax = max(valores) * 1.20  # margen 20%

    ax.set_xlim(0, xmax)

    for i, (val, esp) in enumerate(zip(valores, especies)):
        ax.text(val + xmax * 0.015, i, str(esp), va='center', fontsize=8)

    ax.set_title(titulo, fontsize=20)
    ax.invert_yaxis()
    ax.tick_params(axis='x', labelsize=8)


def procesar_laguna(laguna: str):
    print(f"\n=== Laguna: {laguna} ===")

    dfs = []
    for est in ESTACIONES:
        file_path = INPUT_DIR / f"composicion_{laguna}_{est}.csv"
        if not file_path.exists():
            print(f"  ⚠ No se encuentra {file_path}, se omite {est}.")
            continue

        df = pd.read_csv(file_path)
        if "Especie" not in df.columns or "Abundancia_total" not in df.columns:
            print(f"  ⚠ Formato inesperado en {file_path}, se omite.")
            continue

        df["Estacion"] = est
        dfs.append(df)

    if not dfs:
        print(f"  ❌ No hay datos válidos para {laguna}.")
        return

    data = pd.concat(dfs, ignore_index=True)

    # ---------- Resumen total (todas las estaciones) ----------
# ---------- Resumen total (todas las estaciones) ----------
    resumen_total = (
        data.dropna(subset=["Especie"])
            .groupby("Especie")["Abundancia_total"]
            .sum()
            .reset_index()
    )

    resumen_total = resumen_total[resumen_total["Abundancia_total"] > 0]

    resumen_total["Porcentaje_total"] = (
        resumen_total["Abundancia_total"] /
        resumen_total["Abundancia_total"].sum() * 100
    )

    # ORDENAMOS
    resumen_total = resumen_total.sort_values("Porcentaje_total", ascending=False)

    # 🔥🔥🔥 NUEVA LÍNEA: QUEDARSE SOLO CON EL TOP 10 🔥🔥🔥
    TOP = 10
    resumen_total = resumen_total.head(TOP)

    # Ahora genera listas solo con esas 10 especies
    especies_resumen = resumen_total["Especie"].tolist()
    valores_resumen = resumen_total["Porcentaje_total"].values


    # límite global SOLO para el resumen general
    xmax_global = valores_resumen.max() * 1.20

    # ---------- Figura ----------
    n_rows = 1 + len(ESTACIONES)
    fig, axes = plt.subplots(
        n_rows, 1,
        figsize=(13, 2 + 0.4 * len(especies_resumen) * n_rows),
        sharex=False
    )
    if n_rows == 1:
        axes = [axes]

    # Resumen total (usa xmax_global)
    plot_horizontal_bars(
        axes[0],
        especies_resumen,
        valores_resumen,
        titulo=f"{laguna} - total",
        xmax=xmax_global
    )

    # ---------- Subplots por estación ----------
    for i, est in enumerate(ESTACIONES, start=1):
        ax = axes[i]
        df_est = data[data["Estacion"] == est].copy()

        if df_est.empty:
            ax.set_title(f"{laguna} - {est} (sin datos)",pad=20)
            ax.set_yticks([])
            continue

        est_resume = (
            df_est.dropna(subset=["Especie"])
                  .groupby("Especie")["Abundancia_total"]
                  .sum()
                  .reset_index()
        )
        est_resume = est_resume[est_resume["Abundancia_total"] > 0]

        if est_resume.empty:
            ax.set_title(f"{laguna} - {est} (sin especies)",pad=5)
            ax.set_yticks([])
            continue

        total_est = est_resume["Abundancia_total"].sum()
        est_resume["Porcentaje"] = est_resume["Abundancia_total"] / total_est * 100

        est_resume = est_resume.sort_values("Porcentaje", ascending=False)
        est_resume = est_resume.head(TOP)

        especies_est = est_resume["Especie"].tolist()
        valores_est = est_resume["Porcentaje"].values

        # aquí NO paso xmax → se ajusta solo para cada estación
        plot_horizontal_bars(
            ax,
            especies_est,
            valores_est,
            titulo=f"{laguna} - {est}"
        )

    fig.suptitle(f"Composición porcentual de especies - Laguna {laguna}", y=0.98,fontsize=30)
    plt.tight_layout()
    plt.subplots_adjust(
    hspace=0.4,  # más espacio entre subplots verticales
    top=0.92,   # baja el título principal
    bottom=0.08,
    left=0.1,
    right=0.9
)

    out_path = PLOTS_DIR / f"composicion_especies_{laguna}.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print(f"  ✔ Figura guardada en: {out_path}")


def main():
    for laguna in LAGUNAS:
        procesar_laguna(laguna)
        

if __name__ == "__main__":
    main()


=== Laguna: Termas ===
  ✔ Figura guardada en: plots\composicion_especies_Termas.png


## Generacion de csvs de composiciones totales de especies

In [11]:
# # Directorio donde se guardaron los CSV generados anteriormente
# DATA_DIR = Path("composicion")

# # Estaciones esperadas
# estaciones = ["Invierno", "Verano", "Primavera", "Otoño"]

# # Listado de lagunas
# lagunas = ["Termas", "Viale", "Inta", "Gazano", "Crespo", "Hernandez", "Espinillo"]

# def concatenar_por_laguna(laguna):
#     print(f"\nProcesando laguna: {laguna}")

#     dfs = []  # almacenará los df de cada estación
    
#     for est in estaciones:
#         # Buscar archivos del tipo:
#         # composicion_Termas_Invierno.csv
#         patron = f"composicion_{laguna}_{estaciones}.csv"
#         archivos = list(DATA_DIR.glob(f"composicion_{laguna}_{est}.csv"))

#         if not archivos:
#             print(f"  ⚠️ No existe archivo para {laguna} en {est}. Se omite estación.")
#             continue

#         archivo = archivos[0]  # debería haber solo uno
        
#         df = pd.read_csv(archivo)

#         # Renombrar columna Porcentaje -> estacion_% (ej: Invierno_%)
#         df = df[["Especie", "Porcentaje"]].rename(columns={"Porcentaje": f"{est}_%"})

#         dfs.append(df)

#     if not dfs:
#         print(f"  ❌ No hay datos para {laguna}. Se omite.")
#         return

#     # Merge incremental por "Especie"
#     df_final = dfs[0]
#     for df_est in dfs[1:]:
#         df_final = pd.merge(df_final, df_est, on="Especie", how="outer")

#     # Rellenar NaN con 0
#     df_final = df_final.fillna(0)

#     # Ordenar especies por promedio de porcentajes
#     cols_pct = [c for c in df_final.columns if c.endswith("%")]
#     df_final["Promedio"] = df_final[cols_pct].mean(axis=1)
#     df_final = df_final.sort_values("Promedio", ascending=False).drop(columns=["Promedio"])

#     # Guardar archivo final
#     out_name = f"composicion_total_{laguna}.csv"
#     df_final.to_csv(out_name, index=False)

#     print(f"  ✔ Archivo generado: {out_name}")

# def main():
#     for laguna in lagunas:
#         concatenar_por_laguna(laguna)

# if __name__ == "__main__":
#     main()


## Generacion de cvs dominancias por lagunas

In [12]:
# INPUT_DIR = Path("composicion")      # tus csv individuales
# OUTPUT_DIR = Path("dominancia")        # carpeta nueva para resultados
# OUTPUT_DIR.mkdir(exist_ok=True)

# LAGUNAS = ["Termas", "Viale", "Inta", "Gazano", "Crespo", "Hernandez", "Espinillo"]
# ESTACIONES = ["Invierno", "Verano", "Primavera", "Otoño"]

# def generar_dominancia_laguna(laguna):
#     print(f"\n⏳ Procesando laguna {laguna}...")

#     dfs = []

#     for est in ESTACIONES:
#         file_path = INPUT_DIR / f"composicion_{laguna}_{est}.csv"
#         if not file_path.exists():
#             print(f"  ⚠ No existe {file_path}, se omite.")
#             continue

#         df = pd.read_csv(file_path)
#         if "Especie" not in df.columns or "Abundancia_total" not in df.columns:
#             print(f"  ⚠ Formato incorrecto en {file_path}, se omite.")
#             continue

#         df["Estacion"] = est
#         dfs.append(df)

#     if not dfs:
#         print(f"  ❌ No hay datos válidos para {laguna}")
#         return

#     # Concatenar estaciones
#     data = pd.concat(dfs, ignore_index=True)

#     # Suma total por especie (todas las estaciones)
#     resumen = (
#         data.groupby("Especie")["Abundancia_total"]
#         .sum()
#         .reset_index()
#     )

#     # Filtrar especies presentes
#     resumen = resumen[resumen["Abundancia_total"] > 0]

#     # Calcular porcentaje total
#     total_abundancia = resumen["Abundancia_total"].sum()
#     resumen["Porcentaje_dominancia"] = (resumen["Abundancia_total"] / total_abundancia) * 100

#     # Ordenar
#     resumen = resumen.sort_values("Porcentaje_dominancia", ascending=False)

#     # Guardar
#     out_file = OUTPUT_DIR / f"dominancia_total_{laguna}.csv"
#     resumen.to_csv(out_file, index=False)
#     print(f"  ✔ Archivo guardado: {out_file}")


# def main():
#     for laguna in LAGUNAS:
#         generar_dominancia_laguna(laguna)

# if __name__ == "__main__":
#     main()